Data Import

https://aeturrell.github.io/python4DS/data-import.html

In [2]:
import pandas as pd

In [3]:
students = pd.read_csv("data/students.csv")
students

,Student ID,Full Name,favourite.food,mealPlan,AGE
0,1,Sunil Huffmann,Strawberry yoghurt,Lunch only,4
1,2,Barclay Lynn,French fries,Lunch only,5
2,3,Jayendra Lyne,NaN,Breakfast and lunch,7
3,4,Leon Rossini,Anchovies,Lunch only,8
4,5,Chidiegwu Dunkel,Pizza,Breakfast and lunch,five
5,6,Güvenç Attila,Ice cream,Lunch only,6


The read CSV function automatically creates a new index (which is just the position of each row) and takes the top line of data as the header or column names. But you may wish to tweak this behaviour, in several ways.

Sometimes there are a few lines of metadata at the top of the file. You can use skiprows=n to skip the first n lines, eg pd.read_csv("data/students.csv", skiprows=2).

The data might not have column names. You can use names = a list to tell read_csv() to use a different option for the column names. For example, pd.read_csv("data/students.csv", names=range(5)) would put the numbers 0 to 4 in as the column names.

You may wish to change which column is used as the index. The default behaviour is to create an index, but for this data we see that there already is an ID column we could use. To do this, use the index_col= argument, for example pd.read_csv("data/students.csv", index_col=0).

This is all you need to know to read ~75% of CSV files that you’ll encounter in practice. Reading tab separated files and fixed width files is done with the same function.

Once you read data in, the first step usually involves transforming it in some way to make it easier to work with in the rest of your analysis. For example, the column names in the students file we read in are formatted in non-standard ways.

You might consider renaming them one by one with .rename() or you might use a convenience function from another package to clean them and turn them all into snake case at once. We will make use of the skimpy package to do this. skimpy is a smaller package so isn’t available to install via conda; instead, install it by running pip install skimpy in the terminal.

From skimpy, we will use the clean_columns() function; this takes in a data frame and returns a data frame with variable names converted to snake case.

In [4]:
from skimpy import clean_columns

students = clean_columns(students)
students

,student_id,full_name,favourite_food,meal_plan,age
0,1,Sunil Huffmann,Strawberry yoghurt,Lunch only,4
1,2,Barclay Lynn,French fries,Lunch only,5
2,3,Jayendra Lyne,NaN,Breakfast and lunch,7
3,4,Leon Rossini,Anchovies,Lunch only,8
4,5,Chidiegwu Dunkel,Pizza,Breakfast and lunch,five
5,6,Güvenç Attila,Ice cream,Lunch only,6


Another common task after reading in data is to consider variable types. In the favourite_food column, there are a bunch of food items and then the value NaN, which has been read in as a floating point number rather than a missing string. We can solve this by casting that column to explicitly be composed of strings:

In [5]:
students["favourite_food"] = students["favourite_food"].astype("string")
students

,student_id,full_name,favourite_food,meal_plan,age
0,1,Sunil Huffmann,Strawberry yoghurt,Lunch only,4
1,2,Barclay Lynn,French fries,Lunch only,5
2,3,Jayendra Lyne,<NA>,Breakfast and lunch,7
3,4,Leon Rossini,Anchovies,Lunch only,8
4,5,Chidiegwu Dunkel,Pizza,Breakfast and lunch,five
5,6,Güvenç Attila,Ice cream,Lunch only,6


Similarly, "age" has mixed data types: string and integer! Let’s map the ‘five’ into the number five.


In [6]:
students["age"] = students["age"].replace("five", 5)
students["age"]

0    4
1    5
2    7
3    8
4    5
5    6
Name: age, dtype: object

Another example where the data type is wrong is meal_type. This is a categorical variable with a known set of possible values. pandas has a special data type for these:

In [7]:
students["meal_plan"] = students["meal_plan"].astype("category")
students["meal_plan"]

0             Lunch only
1             Lunch only
2    Breakfast and lunch
3             Lunch only
4    Breakfast and lunch
5             Lunch only
Name: meal_plan, dtype: category
Categories (2, object): ['Breakfast and lunch', 'Lunch only']

It is a bit tedious to have to go through columns one-by-one as single line assignments to apply type. An alternative is to pass a dictionary that maps column names into types, like follows:

In [8]:
students = students.astype({"student_id": "int", "full_name": "string", "age": "int"})
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   student_id      6 non-null      int64   
 1   full_name       6 non-null      string  
 2   favourite_food  5 non-null      string  
 3   meal_plan       6 non-null      category
 4   age             6 non-null      int64   
dtypes: category(1), int64(2), string(2)
memory usage: 454.0 bytes


8.3. Reading data from multiple files
Sometimes your data is split across multiple files instead of being contained in a single file. For example, you might have sales data for multiple months, with each month’s data in a separate file: 01-sales.csv for January, 02-sales.csv for February, and 03-sales.csv for March.

With pd.read_csv() you can read these data in one-by-one and then stack them on top of each other in a single data frame using the pd.concat() function. This looks like:

In [9]:
# list_of_dataframes = [
#     pd.read_csv(x)
#     for x in ["data/01-sales.csv", "data/02-sales.csv", "data/03-sales.csv"]
# ]
# sales_files = pd.concat(list_of_dataframes)
# sales_files

# I don't have the csv's but there's the code above for reference.

If you have many files you want to read in, it can get cumbersome to write out their names as a list. Instead, you can use the glob package (which is built in to Python) to find the files for you by matching a pattern in the file names. Note that there may be other CSV files in the directory data/, so here we specified "*-sales.csv" to ensure we only get those files that include the word sales. Here, "*" plays the role of a wildcard: it represents any series of characters.



In [10]:
import glob

# list_of_csvs = glob.glob("data/*-sales.csv")
# print("List of csvs is:")
# print(list_of_csvs, "\n")
# sales_files = pd.concat([pd.read_csv(x) for x in list_of_csvs])
# sales_files

8.4. Writing to a file
Just as the typical pattern for reading files is pd.read_FILETYPE(), where filetype can be, for example, CSV, all of the ways of writing pandas data frames to disk have the pattern DATAFRAME.to_FILETYPE(). So to write our sales data to a CSV file, the code will be sales_files.to_csv(FILEPATH), where filepath is the location plus name of the file you want to write to.

Let’s see an example of writing data to file using our students data that we already did such good work on setting the data types for:

In [11]:
students.to_csv("data/students-clean.csv")

Now let’s read it back in and check the info on data types:

In [12]:
pd.read_csv("data/students-clean.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      6 non-null      int64 
 1   student_id      6 non-null      int64 
 2   full_name       6 non-null      object
 3   favourite_food  5 non-null      object
 4   meal_plan       6 non-null      object
 5   age             6 non-null      int64 
dtypes: int64(3), object(3)
memory usage: 420.0+ bytes


Notice anything? We lost a lot of the nice data type work we did! While pandas guessed that some columns are integers, we lost the string and categorical variables. The reason for this is that plain text files are not able to carry any contextual information (though pandas will guess some column data types).

If you want to save data in a file and have it remember the data types, you’ll need to use a different data format. For temporary storage, we recommend using the feather format as it is very fast and interoperable with other programming languages. Interoperability is a good reason to avoid language-specific file formats such as Stata’s .dta, R’s .rds, and Python’s .pickle.

Note that the feather format has an additional dependency in the form of a package called pyarrow. To install it, run pip install pyarrow in a terminal window.

Here’s an example of writing to a feather file:

In [13]:
students.to_feather("data/students-clean.feather")